In [ ]:
from src.Evaluation.Evaluator import EvaluatorHoldout
import scipy.sparse as sps
import random
import os
import numpy as np
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

import pandas as pd

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

dataframe_items=pd.read_csv("Dataset/data_ICM_metadata.csv",low_memory=True)
ICM=sps.csr_matrix((dataframe_items["data"].values,(dataframe_items["item_id"].values.astype(int),dataframe_items["feature_id"].values)))

URM_train = sps.load_npz('Dataset/URM_train.npz')
URM_test_complete = sps.load_npz('Dataset/URM_test.npz')

URM_all=URM_train + URM_test_complete

In [ ]:
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
def objective(trial):
    alpha_stacked=trial.suggest_float('alpha_stacked',0.2,1,log=True)
    topk=trial.suggest_int('topk',10,500,log=True)
    alpha=trial.suggest_float('alpha',0.05,1.5,log=True)
    normalize_similarity=trial.suggest_categorical('normalize_similarity',[True,False])
    urm_stacked=sps.vstack([URM_train*alpha_stacked,ICM.T*(1-alpha_stacked)])
    recommender=P3alphaRecommender(urm_stacked)
    recommender.fit(topK=topk,alpha=alpha,implicit=True,normalize_similarity=normalize_similarity)
    result_df, _ = evaluator.evaluateRecommender(recommender)
    return result_df.loc[10, "MAP"]

In [ ]:
import optuna
from optuna.samplers import RandomSampler

database_path='hypertuning_results/tuning_p3aplha_stacked.db'

study=optuna.load_study(
    study_name='hyperparameters_tuning_p3alpha',
    storage=f'sqlite:///{database_path}')
study.sampler=RandomSampler(seed=SEED)

study.optimize(objective, n_trials=900, n_jobs=5)